## Soil vector layer rasterization

This notebook aims at properly rastering the surface depot polygon layer on the same grid as the DEM for use as an input later on with alpine3D.

This not as straightforward as one may think.

In [75]:
import geopandas as gpd
import rasterio
import numpy as np
from rasterio.features import rasterize
from shapely.geometry import Polygon

In [46]:
soil_vector = gpd.read_file('C:/Users/PaulBillecocq/Documents/UdS/KRG_DATA/vecteurs/a3d-depots-surface.gpkg')
soil_vector.head()

,OBJECTID,depot,Shape_Length,Shape_Area,STATUS,House_Numb,Descriptio,Old_Number,SHQ_Housin,SHQ_Buildi,...,Construct1,Source,SHAPE_Leng,Hauteur,Descript_1,KRG_OWNERS,Old_buildi,Demo_Civic,depot_code,geometry
0,1.0,Roc,1541.217149,56890.320474,None,NaN,None,NaN,None,None,...,None,None,NaN,NaN,None,None,None,None,0,"MULTIPOLYGON (((348677.362 6898783.146, 348676..."
1,2.0,Sables,910.166662,19210.165106,None,NaN,None,NaN,None,None,...,None,None,NaN,NaN,None,None,None,None,2,"MULTIPOLYGON (((350013.029 6898753.623, 350022..."
2,3.0,Sables,173.139916,2188.413313,None,NaN,None,NaN,None,None,...,None,None,NaN,NaN,None,None,None,None,2,"MULTIPOLYGON (((349548.451 6898000.090, 349547..."
3,4.0,Sables,116.764776,919.701373,None,NaN,None,NaN,None,None,...,None,None,NaN,NaN,None,None,None,None,2,"MULTIPOLYGON (((349508.850 6898181.318, 349512..."
4,5.0,Sables,77.387010,363.059802,None,NaN,None,NaN,None,None,...,None,None,NaN,NaN,None,None,None,None,2,"MULTIPOLYGON (((349605.870 6897720.657, 349607..."


In [47]:
depot_names = soil_vector.depot.unique().tolist()
# Making sure Hydro is at the right spot with regards to the ALIPNE3D codes (needs to be 01)
depot_names.remove('Hydro')
depot_names.insert(1, 'Hydro')
depot_codes = [i for i in range(len(depot_names))]

In [48]:
mapping_dict = dict(zip(depot_names, depot_codes))

In [49]:
mapping_dict

{'Roc': 0,
 'Hydro': 1,
 'Sables': 2,
 'Argiles': 3,
 'Till': 4,
 'Remblai_Hydro': 5,
 'Remblai_Argiles': 6,
 'Remblai_Sables': 7,
 'Remblai_Roc': 8,
 'Remblai_Till': 9,
 'Batiment_Roc': 10,
 'Batiment_Argiles': 11,
 'Batiment_Sables': 12,
 'Batiment_Till': 13,
 'Batiment_Remblai_Argiles': 14,
 'Batiment_Remblai_Roc': 15,
 'Batiment_Remblai_Till': 16}

In [50]:
soil_vector['depot_code'] = soil_vector.apply(lambda x : mapping_dict[x['depot']], axis=1)

In [76]:
# We need to explode soil_vector because some items are multipolygons
# rasterio's rasterize does not like multipolygons.
soil_vector_exploded = soil_vector.loc[:, ['depot', 'depot_code', 'geometry']].explode(ignore_index=True)
# Turns out also that bulding POLYGONS are actually POLYGONS Z. They need to be converted to 2d as well
# rasterio only likes polygons.
def convert_to_2d(geometry):
    if geometry.has_z:
        return Polygon([(x, y) for x, y, z in geometry.exterior.coords])
    return geometry

soil_vector_exploded['geometry'] = soil_vector_exploded['geometry'].apply(convert_to_2d)

In [78]:
soil_vector_exploded.to_file('C:/Users/PaulBillecocq/Documents/UdS/KRG_DATA/vecteurs/a3d-depots-surface_exploded.gpkg')

### Get DEM to copy the grid for soil depot rasterization

In [85]:
dem = rasterio.open('C:/Users/PaulBillecocq/Documents/UdS/KRG_DATA/ALPINE3D-PREP/DEM/DEM_5m_subgridding_cleaned.tif')
dem_array = dem.read(1)
dem_meta = dem.meta

In [80]:
shapes = list(soil_vector_exploded.apply(lambda x : (x['geometry'], x['depot_code']), axis=1))

In [83]:
rasterized_depots = rasterize(
    shapes=shapes,
    out_shape=(dem_meta['height'], dem_meta['width']),
    transform=dem.transform,
    all_touched=True,
    dtype=rasterio.uint8
)

In [86]:
# Save the rasterized depots
with rasterio.open('C:/Users/PaulBillecocq/Documents/UdS/KRG_DATA/ALPINE3D-PREP/soil-layers/depot_rasterized.tif', 'w', **dem_meta) as out_raster:
    out_raster.write(rasterized_depots, 1)